#Load the Data Set

In [1]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
# Unzip the file
!unzip '/content/drive/MyDrive/DL/CNN_Articels_clean.csv.zip' -d '/content/'

# Load the dataset
df = pd.read_csv('/content/CNN_Articels_clean.csv')

# Check the first few rows of the dataset
print("First few rows of the dataset:")
print(df.head())

# Check for missing values
print("\nMissing values in each column:")
print(df.isnull().sum())

# Get a summary of the data types and non-null counts
print("\nData Summary:")
print(df.info())

# Check the distribution of categories
print("\nDistribution of categories:")
print(df['Category'].value_counts())

Mounted at /content/drive
Archive:  /content/drive/MyDrive/DL/CNN_Articels_clean.csv.zip
  inflating: /content/CNN_Articels_clean.csv  
First few rows of the dataset:
   Index                                             Author  \
0      0                                 Jacopo Prisco, CNN   
1      1                              Stephanie Bailey, CNN   
2      2  Words by Stephanie Bailey, video by Zahra Jamshed   
3      3                                     Kathryn Vasel    
4      4                    Paul R. La Monica, CNN Business   

        Date published  Category    Section  \
0  2021-07-15 02:46:59      news      world   
1  2021-05-12 07:52:09      news      world   
2  2021-06-16 02:51:30      news       asia   
3  2022-03-18 14:37:21  business    success   
4  2022-03-19 11:41:08  business  investing   

                                                 Url  \
0  https://www.cnn.com/2021/07/14/world/tusimple-...   
1  https://www.cnn.com/2021/05/12/world/ironhand-...   
2  

#Clean Data Set

In [2]:
# Drop rows with missing 'Article text'
df_cleaned = df.dropna(subset=['Article text'])

# Verify there are no more missing values in the 'Article text' column
print("\nMissing values after cleaning:")
print(df_cleaned.isnull().sum())


Missing values after cleaning:
Index              0
Author             0
Date published     0
Category           0
Section            0
Url                0
Headline           0
Description        0
Keywords           0
Second headline    0
Article text       0
dtype: int64


In [3]:
import re

# Function to remove the phrase '(CNN)' from the article text
def remove_cnn_phrase(text):
    # Remove variations of (CNN), (cnn), or extra spaces
    return re.sub(r'\(cnn\)|\(CNN\)', '', text).strip()

# Apply the function to the 'Article text' column
df_cleaned['Article text'] = df_cleaned['Article text'].apply(remove_cnn_phrase)

# Check if '(CNN)' was successfully removed
print(df_cleaned['Article text'].head())


0    Right now, there's a shortage of truck drivers...
1    Working in a factory or warehouse can mean doi...
2    In a Hong Kong warehouse, a swarm of autonomou...
3    The pandemic thrust the working world into a n...
4    New York (CNN Business)March Madness isn't jus...
Name: Article text, dtype: object


<ipython-input-3-d6b8de0340dc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Article text'] = df_cleaned['Article text'].apply(remove_cnn_phrase)


In [4]:
# Convert 'Date published' to datetime format
df_cleaned['Date published'] = pd.to_datetime(df_cleaned['Date published'], errors='coerce')

# Check for any invalid dates that couldn't be converted
print("\nInvalid dates after conversion:")
print(df_cleaned[df_cleaned['Date published'].isnull()])

# Drop rows with invalid dates, if necessary
df_cleaned = df_cleaned.dropna(subset=['Date published'])

# Check final shape of the cleaned dataset
print("\nFinal dataset shape after cleaning:")
print(df_cleaned.shape)


Invalid dates after conversion:
Empty DataFrame
Columns: [Index, Author, Date published, Category, Section, Url, Headline, Description, Keywords, Second headline, Article text]
Index: []

Final dataset shape after cleaning:
(37940, 11)


<ipython-input-4-c97cd517248b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Date published'] = pd.to_datetime(df_cleaned['Date published'], errors='coerce')


In [ ]:
# Save the cleaned dataset to a new CSV file
df_cleaned.to_csv('data/cnn_articles_cleaned.csv', index=False)

#Data Preprocessing

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords from NLTK
nltk.download('stopwords')
nltk.download('punkt')

# Define stop words
stop_words = set(stopwords.words('english'))

# Function for text preprocessing
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()

    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stop words
    words = [word for word in words if word not in stop_words]

    # Join the words back into a single string
    return ' '.join(words)

# Apply preprocessing to the 'Article text' column
df_cleaned['cleaned_text'] = df_cleaned['Article text'].apply(preprocess_text)

# Check the first few rows of the cleaned text
print(df_cleaned[['Article text', 'cleaned_text']].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


                                        Article text  \
0  Right now, there's a shortage of truck drivers...   
1  Working in a factory or warehouse can mean doi...   
2  In a Hong Kong warehouse, a swarm of autonomou...   
3  The pandemic thrust the working world into a n...   
4  New York (CNN Business)March Madness isn't jus...   

                                        cleaned_text  
0  right theres shortage truck drivers us worldwi...  
1  working factory warehouse mean task repetition...  
2  hong kong warehouse swarm autonomous robots wo...  
3  pandemic thrust working world new reality marc...  
4  new york cnn businessmarch madness isnt colleg...  


#Tokenization and Padding

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Define the maximum number of words to use (top 20,000 most frequent words)
max_words = 20000
# Define the maximum length of a sequence (for padding)
max_len = 100

# Initialize the tokenizer and fit on the cleaned text
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_cleaned['cleaned_text'])

# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences(df_cleaned['cleaned_text'])

# Apply padding to ensure all sequences have the same length
X = pad_sequences(sequences, maxlen=max_len)

# Convert categories to integer labels (for classification)
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_cleaned['Category'])

#New
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shapes of the final processed data
print("Shape of X (features):", X.shape)
print("Shape of y (labels):", y.shape)

Shape of X (features): (37940, 100)
Shape of y (labels): (37940,)


#Build LSTM Model

In [9]:
#Number of unique news categories.

unique_category_count = df['Category'].nunique()
print(f"Number of unique categories: {unique_category_count}")

Number of unique categories: 9


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, BatchNormalization, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the LSTM model
model = Sequential()

# Embedding layer
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len, input_shape=(max_len,)))

# LSTM layer with L2 regularization and Dropout
model.add(LSTM(128, activation='relu', kernel_regularizer=l2(0.01), return_sequences=True))
model.add(Dropout(0.5))

# Batch normalization
model.add(BatchNormalization())

# LSTM layer with L2 regularization
model.add(LSTM(64, activation='tanh', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))

# Batch normalization
model.add(BatchNormalization())

# Output layer with L2 regularization
model.add(Dense(unique_category_count, activation='softmax', kernel_regularizer=l2(0.01)))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0005)

#model summary
model.summary()

# Train the model
# history = model.fit(X, y, epochs=20, batch_size=64, validation_split=0.2, callbacks=[early_stopping, reduce_lr])
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2, callbacks=[early_stopping, reduce_lr])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 128)            │       2,560,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100, 128)            │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 100, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 9)                   │             585 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,742,345 (10.46 MB)

 Trainable params: 2,741,961 (10.46 MB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 81s 188ms/step - accuracy: 0.7065 - loss: 2.7677 - val_accuracy: 0.7671 - val_loss: 1.1396 - learning_rate: 0.0010
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 73s 182ms/step - accuracy: 0.9013 - loss: 0.5787 - val_accuracy: 0.8824 - val_loss: 0.5245 - learning_rate: 0.0010
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 84s 189ms/step - accuracy: 0.9252 - loss: 0.3618 - val_accuracy: 0.8702 - val_loss: 0.5339 - learning_rate: 0.0010
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 79s 181ms/step - accuracy: 0.9380 - loss: 0.2926 - val_accuracy: 0.8778 - val_loss: 0.5007 - learning_rate: 0.0010
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 74s 194ms/step - accuracy: 0.9418 - loss: 0.2654 - val_accuracy: 0.8908 - val_loss: 0.5122 - learning_rate: 0.0010
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 79s 187ms/step - accuracy: 0.9483 - loss: 0.2329 - val_accuracy: 0.8542 - val_loss: 0.5982 - learning_rate: 0.0010
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 73s 191ms/step - accuracy: 0.9